System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin21.5.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, apple-m1)
  Threads: 8 on 8 virtual cores
Environment:
  JULIA_NUM_THREADS = 8
  JULIA_EDITOR = code


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/github.com/ucla-biostat-257/2023spring/slides/08-numalgintro`


Status `~/Documents/github.com/ucla-biostat-257/2023spring/slides/08-numalgintro/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [0e44f5e4] Hwloc v2.2.0
  [bdcacae8] LoopVectorization v0.12.157
  [6f49c342] RCall v0.13.14
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random


## Introduction

* Topics in numerical algebra: 
    - BLAS  
    - solve linear equations $\mathbf{A} \mathbf{x} = \mathbf{b}$
    - regression computations $\mathbf{X}^T \mathbf{X} \beta = \mathbf{X}^T \mathbf{y}$  
    - eigen-problems $\mathbf{A} \mathbf{x} = \lambda \mathbf{x}$  
    - generalized eigen-problems $\mathbf{A} \mathbf{x} = \lambda \mathbf{B} \mathbf{x}$  
    - singular value decompositions $\mathbf{A} = \mathbf{U} \Sigma \mathbf{V}^T$  
    - iterative methods for numerical linear algebra    

* Except for the iterative methods, most of these numerical linear algebra tasks are implemented in the **BLAS** and **LAPACK** libraries. They form the **building blocks** of most statistical computing tasks (optimization, MCMC).

* Our major **goal** (or learning objectives) is to  
    1. know the complexity (flop count) of each task
    2. be familiar with the BLAS and LAPACK functions (what they do)  
    3. do **not** re-invent wheels by implementing these dense linear algebra subroutines by yourself  
    4. understand the need for iterative methods  
    5. apply appropriate numerical algebra tools to various statistical problems 

* All high-level languages (Julia, Matlab, Python, R) call BLAS and LAPACK for numerical linear algebra. 
    - Julia offers more flexibility by exposing interfaces to many BLAS/LAPACK subroutines directly. See [documentation](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#BLAS-functions-1).

## BLAS

* BLAS stands for _basic linear algebra subprograms_. 

* See [netlib](http://www.netlib.org/blas/) for a complete list of standardized BLAS functions.

* There are many implementations of BLAS. 
    - [Netlib](http://www.netlib.org/blas/) provides a reference implementation.  
    - Matlab uses Intel's [MKL](https://software.intel.com/en-us/node/520724) (mathematical kernel libaries). **MKL implementation is the gold standard on market.** It is not open source but the compiled library is free for Linux and MacOS.    
    - Julia uses [OpenBLAS](https://github.com/xianyi/OpenBLAS). **OpenBLAS is the best open source implementation**.    

* There are 3 levels of BLAS functions.
    - [Level 1](http://www.netlib.org/blas/#_level_1): vector-vector operation
    - [Level 2](http://www.netlib.org/blas/#_level_2): matrix-vector operation
    - [Level 3](http://www.netlib.org/blas/#_level_3): matrix-matrix operation

| Level | Example Operation                      | Name        | Dimension                                 | Flops |  
|-------|----------------------------------------|-------------|-------------------------------------------|-------|
| 1     | $\alpha \gets \mathbf{x}^T \mathbf{y}$ | dot product | $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$ | $2n$  |  
| 1 | $\mathbf{y} \gets \mathbf{y} + \alpha \mathbf{x}$ |  axpy           |  $\alpha \in \mathbb{R}$, $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$ |  $2n$    |  
| 2     | $\mathbf{y} \gets \mathbf{y} + \mathbf{A} \mathbf{x}$ |  gaxpy           |  $\mathbf{A} \in \mathbb{R}^{m \times n}$, $\mathbf{x} \in \mathbb{R}^n$, $\mathbf{y} \in \mathbb{R}^m$                                     |  $2mn$     |
| 2 | $\mathbf{A} \gets \mathbf{A} + \mathbf{y} \mathbf{x}^T$ | rank one update            |    $\mathbf{A} \in \mathbb{R}^{m \times n}$, $\mathbf{x} \in \mathbb{R}^n$, $\mathbf{y} \in \mathbb{R}^m$                                       | $2mn$      |
| 3     | $\mathbf{C} \gets \mathbf{C} + \mathbf{A} \mathbf{B}$                                       |  matrix multiplication           |  $\mathbf{A} \in \mathbb{R}^{m \times p}$, $\mathbf{B} \in \mathbb{R}^{p \times n}$, $\mathbf{C} \in \mathbb{R}^{m \times n}$                                         | $2mnp$      |

* Typical BLAS functions support single precision (S), double precision (D), complex (C), and double complex (Z). 

## Examples

> **The form of a mathematical expression and the way the expression should be evaluated in actual practice may be quite different.**

Some operations _appear_ as level-3 but indeed are level-2.  

**Example 1**. A common operation in statistics is column scaling or row scaling
$$
\begin{eqnarray*}
    \mathbf{A} &=& \mathbf{A} \mathbf{D} \quad \text{(column scaling)} \\
    \mathbf{A} &=& \mathbf{D} \mathbf{A} \quad \text{(row scaling)},
\end{eqnarray*}
$$
where $\mathbf{D}$ is diagonal. For example, in generalized linear models (GLMs), the Fisher information matrix takes the form  
$$
\mathbf{X}^T \mathbf{W} \mathbf{X},
$$
where $\mathbf{W}$ is a diagonal matrix with observation weights on diagonal.  

  Column and row scalings are essentially level-2 operations!

In [3]:
using BenchmarkTools, LinearAlgebra, Random

Random.seed!(123) # seed
n = 2000
A = rand(n, n) # n-by-n matrix
d = rand(n)  # n vector
D = Diagonal(d) # diagonal matrix with d as diagonal

2000×2000 Diagonal{Float64, Vector{Float64}}:
 0.919181   ⋅         ⋅         ⋅        …   ⋅         ⋅          ⋅ 
  ⋅        0.426019   ⋅         ⋅            ⋅         ⋅          ⋅ 
  ⋅         ⋅        0.746586   ⋅            ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅        0.819201      ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅            ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅        …   ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅            ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅            ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅            ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅            ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅        …   ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅            ⋅         ⋅          ⋅ 
  ⋅         ⋅         ⋅         ⋅            ⋅         ⋅          ⋅ 
 ⋮                                       ⋱               

In [4]:
Dfull = diagm(d) # convert to full matrix

2000×2000 Matrix{Float64}:
 0.919181  0.0       0.0       0.0       …  0.0       0.0        0.0
 0.0       0.426019  0.0       0.0          0.0       0.0        0.0
 0.0       0.0       0.746586  0.0          0.0       0.0        0.0
 0.0       0.0       0.0       0.819201     0.0       0.0        0.0
 0.0       0.0       0.0       0.0          0.0       0.0        0.0
 0.0       0.0       0.0       0.0       …  0.0       0.0        0.0
 0.0       0.0       0.0       0.0          0.0       0.0        0.0
 0.0       0.0       0.0       0.0          0.0       0.0        0.0
 0.0       0.0       0.0       0.0          0.0       0.0        0.0
 0.0       0.0       0.0       0.0          0.0       0.0        0.0
 0.0       0.0       0.0       0.0       …  0.0       0.0        0.0
 0.0       0.0       0.0       0.0          0.0       0.0        0.0
 0.0       0.0       0.0       0.0          0.0       0.0        0.0
 ⋮                                       ⋱                       
 0.0      

In [5]:
# this is calling BLAS routine for matrix multiplication: O(n^3) flops
# this is SLOW!
@benchmark $A * $Dfull

BenchmarkTools.Trial: 100 samples with 1 evaluation.
 Range (min … max):  46.749 ms … 121.329 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     47.998 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   50.097 ms ±   9.852 ms  ┊ GC (mean ± σ):  0.84% ± 1.48%

  █▇▅                                                           
  ████▅▁▁▅▅▁▁▅▅▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅ ▅
  46.7 ms       Histogram: log(frequency) by time       103 ms <

 Memory estimate: 30.52 MiB, allocs estimate: 2.

In [6]:
# dispatch to special method for diagonal matrix multiplication.
# columnwise scaling: O(n^2) flops
@benchmark $A * $D

BenchmarkTools.Trial: 1856 samples with 1 evaluation.
 Range (min … max):  2.178 ms …   4.405 ms  ┊ GC (min … max):  0.00% … 23.72%
 Time  (median):     2.418 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.692 ms ± 521.500 μs  ┊ GC (mean ± σ):  10.25% ± 13.50%

    ▂▅▆█▆▃                                                     
  ▃▅███████▆▆▆▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▂▁▁▂▁▁▁▁▂▂▄▄▄▅▆▅▅▅▄▃▃▃▃▃▃▃▃▃▃▂▂▂ ▃
  2.18 ms         Histogram: frequency by time        3.97 ms <

 Memory estimate: 30.52 MiB, allocs estimate: 2.

In [7]:
# in-place: avoid allocate space for result
# rmul!: compute matrix-matrix product AB, overwriting A, and return the result.
@benchmark rmul!($A, $D)

BenchmarkTools.Trial: 8690 samples with 1 evaluation.
 Range (min … max):  523.750 μs …  2.977 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     568.708 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   572.344 μs ± 30.284 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                           █                                    
  ▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▃██▆█▅▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  524 μs          Histogram: frequency by time          613 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

**Note:** In R or Matlab, `diag(d)` will create a full matrix. Be cautious using `diag` function: do we really need a full diagonal matrix?

In [8]:
using RCall

R"""
d <- runif(5)
diag(d)
"""

RObject{RealSxp}
          [,1]      [,2]      [,3]      [,4]      [,5]
[1,] 0.3571382 0.0000000 0.0000000 0.0000000 0.0000000
[2,] 0.0000000 0.2755224 0.0000000 0.0000000 0.0000000
[3,] 0.0000000 0.0000000 0.8956138 0.0000000 0.0000000
[4,] 0.0000000 0.0000000 0.0000000 0.7624779 0.0000000
[5,] 0.0000000 0.0000000 0.0000000 0.0000000 0.7368475


In [9]:
#| eval: false

# This works only when Matlab is installed
using MATLAB

mat"""
d = rand(5, 1)
diag(d)
"""

LoadError: ArgumentError: Package MATLAB not found in current path.
- Run `import Pkg; Pkg.add("MATLAB")` to install the MATLAB package.

**Example 2**. Innter product between two matrices $\mathbf{A}, \mathbf{B} \in \mathbb{R}^{m \times n}$ is often written as 
$$
    \text{trace}(\mathbf{A}^T \mathbf{B}), \text{trace}(\mathbf{B} \mathbf{A}^T), \text{trace}(\mathbf{A} \mathbf{B}^T), \text{ or } \text{trace}(\mathbf{B}^T \mathbf{A}).
$$
They appear as level-3 operation (matrix multiplication with $O(m^2n)$ or $O(mn^2)$ flops).

In [10]:
Random.seed!(123)
n = 2000
A, B = randn(n, n), randn(n, n)

# slow way to evaluate tr(A'B)
@benchmark tr(transpose($A) * $B)

BenchmarkTools.Trial: 105 samples with 1 evaluation.
 Range (min … max):  46.818 ms … 51.360 ms  ┊ GC (min … max): 0.00% … 3.67%
 Time  (median):     47.512 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   47.923 ms ±  1.091 ms  ┊ GC (mean ± σ):  0.97% ± 1.65%

   ▂▇█▃                                                        
  ▃████▄▃▁▆▁▄▇▅▅▃▄▁▃▁▃▁▁▁▁▃▁▁▁▃▁▁▇▇▅▄▃▁▃▃▄▁▁▅▅▄▁▃▁▁▃▁▁▁▃▁▁▁▁▃ ▃
  46.8 ms         Histogram: frequency by time        50.6 ms <

 Memory estimate: 30.52 MiB, allocs estimate: 2.

But $\text{trace}(\mathbf{A}^T \mathbf{B}) = <\text{vec}(\mathbf{A}), \text{vec}(\mathbf{B})>$. The latter is level-1 BLAS operation with $O(mn)$ flops.

In [11]:
# smarter way to evaluate tr(A'B)
@benchmark dot($A, $B)

BenchmarkTools.Trial: 2703 samples with 1 evaluation.
 Range (min … max):  1.768 ms …  2.035 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.843 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.847 ms ± 41.574 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▁▁ ▃▂▅▄▃▄▄█▅▆▆▆▆▆▃▄▃▃▄▄▄▄▄▁▂▁                          
  ▂▂▃▆▇█████████████████████████████▇▇▆▆▅▄▅▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▁▂ ▅
  1.77 ms        Histogram: frequency by time        1.97 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

**Example 3**. Similarly $\text{diag}(\mathbf{A}^T \mathbf{B})$ can be calculated in $O(mn)$ flops.

In [12]:
# slow way to evaluate diag(A'B): O(n^3)
@benchmark diag(transpose($A) * $B)

BenchmarkTools.Trial: 105 samples with 1 evaluation.
 Range (min … max):  46.730 ms … 52.680 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     47.546 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   48.003 ms ±  1.156 ms  ┊ GC (mean ± σ):  0.97% ± 1.66%

  ▃▂▃    ▂ █▇                                                  
  ███▆▃▆▁████▁▆▆▆▃▃▅▁▃▃▁▁▁▃▁▃▆▆▁▆▇▁▇▆▃▅▁▃▁▁▁▁▁▁▃▁▃▁▁▁▁▁▁▁▁▁▁▅ ▃
  46.7 ms         Histogram: frequency by time        51.4 ms <

 Memory estimate: 30.53 MiB, allocs estimate: 3.

In [13]:
# smarter way to evaluate diag(A'B): O(n^2)
@benchmark Diagonal(vec(sum($A .* $B, dims=1)))

BenchmarkTools.Trial: 1305 samples with 1 evaluation.
 Range (min … max):  3.258 ms …   5.878 ms  ┊ GC (min … max): 0.00% … 33.28%
 Time  (median):     3.487 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.826 ms ± 641.034 μs  ┊ GC (mean ± σ):  9.05% ± 12.34%

    ▇█▃                                                        
  ▂▆███▇▆▅▄▄▄▄▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▂▂▂▅▇▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▃▂▃ ▃
  3.26 ms         Histogram: frequency by time        5.66 ms <

 Memory estimate: 30.53 MiB, allocs estimate: 5.

To get rid of allocation of intermediate arrays at all, we can just write a double loop or use `dot` function.

In [14]:
using LoopVectorization

function diag_matmul!(d, A, B)
    m, n = size(A)
    @assert size(B) == (m, n) "A and B should have same size"
    fill!(d, 0)
    @turbo for j in 1:n, i in 1:m
        d[j] += A[i, j] * B[i, j]
    end
    Diagonal(d)
end

d = zeros(eltype(A), size(A, 2))
@benchmark diag_matmul!($d, $A, $B)

BenchmarkTools.Trial: 3565 samples with 1 evaluation.
 Range (min … max):  1.354 ms …  1.748 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.390 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.397 ms ± 34.182 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▂▄▇██▇▇▆▆▆▅▅▂▁▁▁▁▁                                        
  ▃▅██████████████████▇▇▇▆▅▆▅▄▅▅▃▄▃▃▃▂▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂ ▄
  1.35 ms        Histogram: frequency by time        1.53 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

## Memory hierarchy and level-3 fraction

> **Key to high performance is effective use of memory hierarchy. True on all architectures.**

* Flop count is not the sole determinant of algorithm efficiency. Another important factor is data movement through the memory hierarchy.

<img src="./macpro_inside.png" width="400" align="center">

<img src="./cpu_die.png" width="400" align="center">  

<img src="./hei.png" width="400" align="center">

* Numbers everyone should know

| Operation                           | Time           |
|-------------------------------------|----------------|
| L1 cache reference                  | 0.5 ns         |
| L2 cache reference                  | 7 ns           |
| Main memory reference               | 100 ns         |
| Read 1 MB sequentially from memory  | 250,000 ns     |
| Read 1 MB sequentially from SSD     | 1,000,000 ns   |  
| Read 1 MB sequentially from disk    | 20,000,000 ns  |


<!-- | Operation                           | Time           | -->
<!-- |-------------------------------------|----------------| -->
<!-- | L1 cache reference                  | 0.5 ns         | -->
<!-- | Branch mispredict                   | 5 ns           | -->
<!-- | L2 cache reference                  | 7 ns           | -->
<!-- | Mutex lock/unlock                   | 100 ns         | -->
<!-- | Main memory reference               | 100 ns         | -->
<!-- | Compress 1K bytes with Zippy        | 10,000 ns      | -->
<!-- | Send 2K bytes over 1 Gbps network   | 20,000 ns      | -->
<!-- | Read 1 MB sequentially from memory  | 250,000 ns     | -->
<!-- | Round trip within same datacenter   | 500,000 ns     | -->
<!-- | Disk seek                           | 10,000,000 ns  | -->
<!-- | Read 1 MB sequentially from network | 10,000,000 ns  | -->
<!-- | Read 1 MB sequentially from disk    | 30,000,000 ns  | -->
<!-- | Send packet CA->Netherlands->CA     | 150,000,000 ns | -->

   Source: <https://gist.github.com/jboner/2841832>  

- In Julia, we can query the CPU memory hierarchy by the `Hwloc.jl` package.

In [15]:
using Hwloc

topology_graphical()

┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Machine (1975MB total)                                                                                                                     │
│                                                                                                                                            │
│ ┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐ │
│ │ Package L#0                                                                                                                            │ │
│ │                                                                                                                                        │ │
│ │ ┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐ │ │

* For example, Xeon X5650 CPU has a theoretical throughput of 128 DP GFLOPS but a max memory bandwidth of 32GB/s.  

* Can we keep CPU cores busy with enough deliveries of matrix data and ship the results to memory fast enough to avoid backlog?  
Answer: use **high-level BLAS** as much as possible.

| BLAS | Dimension | Mem. Refs. | Flops  | Ratio |
|--------------------------------|------------------------------------------------------------|------------|--------|-------|
| Level 1: $\mathbf{y} \gets \mathbf{y} + \alpha \mathbf{x}$     | $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$                                           | $3n$       | $2n$   | 3:2   |
| Level 2: $\mathbf{y} \gets \mathbf{y} + \mathbf{A} \mathbf{x}$ | $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$, $\mathbf{A} \in \mathbb{R}^{n \times n}$ | $n^2$      | $2n^2$ | 1:2   |
| Level 3: $\mathbf{C} \gets \mathbf{C} + \mathbf{A} \mathbf{B}$ | $\mathbf{A}, \mathbf{B}, \mathbf{C} \in\mathbb{R}^{n \times n}$                    | $4n^2$     | $2n^3$ | 2:n |  

* Higher level BLAS (3 or 2) make more effective use of arithmetic logic units (ALU) by keeping them busy. **Surface-to-volume** effect.  
See [Dongarra slides](https://www.samsi.info/wp-content/uploads/2017/02/SAMSI-0217_Dongarra.pdf).

<img src="./blas_throughput.png" width="500" align="center"/>

* A distinction between LAPACK and LINPACK (older version of R uses LINPACK) is that LAPACK makes use of higher level BLAS as much as possible (usually by smart partitioning) to increase the so-called **level-3 fraction**.

* To appreciate the efforts in an optimized BLAS implementation such as OpenBLAS (evolved from GotoBLAS), see the [Quora question](https://www.quora.com/What-algorithm-does-BLAS-use-for-matrix-multiplication-Of-all-the-considerations-e-g-cache-popular-instruction-sets-Big-O-etc-which-one-turned-out-to-be-the-primary-bottleneck), especially the [video](https://youtu.be/JzNpKDW07rw). Bottomline is 

> **Get familiar with (good implementations of) BLAS/LAPACK and use them as much as possible.**

## Effect of data layout

* Data layout in memory affects algorithmic efficiency too. It is much faster to move chunks of data in memory than retrieving/writing scattered data.

* Storage mode: **column-major** (Fortran, Matlab, R, Julia) vs **row-major** (C/C++).

* **Cache line** is the minimum amount of cache which can be loaded and stored to memory.
    - x86 CPUs: 64 bytes  
    - ARM CPUs: 32 bytes

<img src="https://patterns.eecs.berkeley.edu/wordpress/wp-content/uploads/2013/04/dense02.png" width="500" align="center"/>

* In Julia, we can query the cache line size by `Hwloc.

In [16]:
# Apple Silicon (M1/M2 chips) don't have L3 cache
Hwloc.cachelinesize()

LoadError: Your system doesn't seem to have an L3 cache.

* Accessing column-major stored matrix by rows ($ij$ looping) causes lots of **cache misses**.

* Take matrix multiplication as an example 
$$ 
\mathbf{C} \gets \mathbf{C} + \mathbf{A} \mathbf{B}, \quad \mathbf{A} \in \mathbb{R}^{m \times p}, \mathbf{B} \in \mathbb{R}^{p \times n}, \mathbf{C} \in \mathbb{R}^{m \times n}.
$$
Assume the storage is column-major, such as in Julia. There are 6 variants of the algorithms according to the order in the triple loops. 

    - `jki` or `kji` looping:
    
```julia
# inner most loop
for i in 1:m
    C[i, j] = C[i, j] + A[i, k] * B[k, j]
end
```
        
    - `ikj` or `kij` looping:

```julia
# inner most loop        
for j in 1:n
    C[i, j] = C[i, j] + A[i, k] * B[k, j]
end
```  

- `ijk` or `jik` looping:

```julia
# inner most loop        
for k in 1:p
    C[i, j] = C[i, j] + A[i, k] * B[k, j]
end
```
        
* We pay attention to the innermost loop, where the vector calculation occurs. The associated **stride** when accessing the three matrices in memory (assuming column-major storage) is  

| Variant        | A Stride | B Stride | C Stride |
|----------------|----------|----------|----------|
| $jki$ or $kji$ | Unit     | 0        | Unit     |
| $ikj$ or $kij$ | 0        | Non-Unit | Non-Unit |
| $ijk$ or $jik$ | Non-Unit | Unit     | 0        |       
Apparently the variants $jki$ or $kji$ are preferred.

In [17]:
"""
    matmul_by_loop!(A, B, C, order)

Overwrite `C` by `A * B`. `order` indicates the looping order for triple loop.
"""
function matmul_by_loop!(A::Matrix, B::Matrix, C::Matrix, order::String)
    
    m = size(A, 1)
    p = size(A, 2)
    n = size(B, 2)
    fill!(C, 0)
    
    if order == "jki"
        @inbounds for j = 1:n, k = 1:p, i = 1:m
            C[i, j] += A[i, k] * B[k, j]
        end
    end

    if order == "kji"
        @inbounds for k = 1:p, j = 1:n, i = 1:m
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
    if order == "ikj"
        @inbounds for i = 1:m, k = 1:p, j = 1:n
            C[i, j] += A[i, k] * B[k, j]
        end
    end

    if order == "kij"
        @inbounds for k = 1:p, i = 1:m, j = 1:n
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
    if order == "ijk"
        @inbounds for i = 1:m, j = 1:n, k = 1:p
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
    if order == "jik"
        @inbounds for j = 1:n, i = 1:m, k = 1:p
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
end

using Random

Random.seed!(123)
m, p, n = 2000, 100, 2000
A = rand(m, p)
B = rand(p, n)
C = zeros(m, n);

* $jki$ and $kji$ looping:

In [18]:
using BenchmarkTools

@benchmark matmul_by_loop!($A, $B, $C, "jki")

BenchmarkTools.Trial: 86 samples with 1 evaluation.
 Range (min … max):  57.911 ms …  59.010 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     58.297 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   58.309 ms ± 313.182 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▂                                                            
  ███▅▁▁▃▁▆▁▅▃▁▁▁▃▆▁▁▅▃▃▅▅▃▃▃▅▁▃▃▃▆▁▁▃▅▃▇▃▁▁▁▆▁▆▃▅▆▅▆▃▁▁▃▃▁▃▁▃ ▁
  57.9 ms         Histogram: frequency by time         58.9 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [19]:
@benchmark matmul_by_loop!($A, $B, $C, "kji")

BenchmarkTools.Trial: 27 samples with 1 evaluation.
 Range (min … max):  184.098 ms … 188.095 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     186.687 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   186.538 ms ±   1.204 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁   ▁       ▁     █▁ █  █      ▁   ▁ ▁ ▁▁      ▁▁  █▁▁ ▁▁ █ █  
  █▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁██▁█▁▁█▁▁▁▁▁▁█▁▁▁█▁█▁██▁▁▁▁▁▁██▁▁███▁██▁█▁█ ▁
  184 ms           Histogram: frequency by time          188 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

* $ikj$ and $kij$ looping:

In [20]:
@benchmark matmul_by_loop!($A, $B, $C, "ikj")

BenchmarkTools.Trial: 10 samples with 1 evaluation.
 Range (min … max):  511.047 ms … 518.636 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     515.570 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   514.741 ms ±   2.656 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █   ▁                    ▁      ▁       ▁ ▁ ▁▁              ▁  
  █▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁█▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  511 ms           Histogram: frequency by time          519 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [21]:
@benchmark matmul_by_loop!($A, $B, $C, "kij")

BenchmarkTools.Trial: 10 samples with 1 evaluation.
 Range (min … max):  504.824 ms … 507.888 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     505.601 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   505.840 ms ± 850.075 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁     ▁    ▁█    ▁      ▁▁   ▁                              ▁  
  █▁▁▁▁▁█▁▁▁▁██▁▁▁▁█▁▁▁▁▁▁██▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  505 ms           Histogram: frequency by time          508 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

* $ijk$ and $jik$ looping:

In [22]:
@benchmark matmul_by_loop!($A, $B, $C, "ijk")

BenchmarkTools.Trial: 22 samples with 1 evaluation.
 Range (min … max):  234.292 ms … 242.526 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     235.570 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   236.077 ms ±   2.168 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                              
  █▁▅▅▅▁█▁▁█▅▅█▁▁▁▅▁▁▅▅▁▁▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁▁▁▅ ▁
  234 ms           Histogram: frequency by time          243 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [23]:
@benchmark matmul_by_loop!($A, $B, $C, "ijk")

BenchmarkTools.Trial: 22 samples with 1 evaluation.
 Range (min … max):  234.247 ms … 241.833 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     235.162 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   235.822 ms ±   1.877 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▁▁    ▁                                                       
  ███▁▆▁▆█▆▁▆▁▁▁▁▆▁▆▁▆▆▁▁▁▁▆▁▆▁▁▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  234 ms           Histogram: frequency by time          242 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

* **Question: Can our loop beat BLAS?** Julia wraps BLAS library for matrix multiplication. We see BLAS library wins hands down (multi-threading, Strassen algorithm, higher level-3 fraction by block outer product).

In [24]:
@benchmark mul!($C, $A, $B)

BenchmarkTools.Trial: 1830 samples with 1 evaluation.
 Range (min … max):  2.701 ms …  3.898 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.726 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.730 ms ± 33.379 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

            ▁▃▇█▇▆▆▅ ▁                                        
  ▂▁▂▂▂▂▂▂▃▅██████████▇▆▆▄▅▅▄▄▄▃▃▃▃▃▃▂▃▃▂▂▂▂▃▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂ ▃
  2.7 ms         Histogram: frequency by time        2.79 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [25]:
# direct call of BLAS wrapper function
@benchmark LinearAlgebra.BLAS.gemm!('N', 'N', 1.0, $A, $B, 0.0, $C)

BenchmarkTools.Trial: 1881 samples with 1 evaluation.
 Range (min … max):  2.637 ms …   6.744 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.650 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.655 ms ± 101.620 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

          ▄▅▆██▆▆▄▃▂                                           
  ▂▂▂▃▄▆▇▇████████████▇▆▆▅▅▄▃▄▄▃▃▃▃▂▃▂▂▂▃▂▂▂▂▂▁▂▂▂▁▂▁▂▂▁▂▂▁▂▂ ▄
  2.64 ms         Histogram: frequency by time        2.69 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

**Question (again): Can our loop beat BLAS?**

**Exercise:** Annotate the loop in `matmul_by_loop!` by `@turbo` and `@tturbo` (multi-threading) and benchmark again.

## BLAS in R

* **Tip for R users**. Standard R distribution from CRAN uses a very out-dated BLAS/LAPACK library.

In [26]:
using RCall

R"""
sessionInfo()
"""

RObject{VecSxp}
R version 4.2.2 (2022-10-31)
Platform: aarch64-apple-darwin20 (64-bit)
Running under: macOS Ventura 13.3.1

Matrix products: default
LAPACK: /Library/Frameworks/R.framework/Versions/4.2-arm64/Resources/lib/libRlapack.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
[1] compiler_4.2.2


In [27]:
R"""
library(dplyr)
library(bench)
A <- $A
B <- $B
bench::mark(A %*% B) %>%
  print(width = Inf)
""";

┌ Warning: RCall.jl: 
│ Attaching package: ‘dplyr’
│ 
│ The following objects are masked from ‘package:stats’:
│ 
│     filter, lag
│ 
│ The following objects are masked from ‘package:base’:
│ 
│     intersect, setdiff, setequal, union
│ 
└ @ RCall ~/.julia/packages/RCall/Wyd74/src/io.jl:172


# A tibble: 1 × 13
  expression      min   median `itr/sec` mem_alloc `gc/sec` n_itr  n_gc
  <bch:expr> <bch:tm> <bch:tm>     <dbl> <bch:byt>    <dbl> <int> <dbl>
1 A %*% B       122ms    124ms      7.78    30.5MB     7.78     4     4
  total_time result                memory             time          
    <bch:tm> <list>                <list>             <list>        
1      514ms <dbl [2,000 × 2,000]> <Rprofmem [1 × 3]> <bench_tm [4]>
  gc              
  <list>          
1 <tibble [4 × 3]>


┌ Warning: RCall.jl: Warning: Some expressions had a GC in every iteration; so filtering is disabled.
└ @ RCall ~/.julia/packages/RCall/Wyd74/src/io.jl:172


* Re-build R from source using OpenBLAS or MKL will immediately boost linear algebra performance in R. Google `build R using MKL` to get started. Similarly we can build Julia using MKL.

* Matlab uses MKL. Usually it's very hard to beat Matlab in terms of linear algebra.

In [28]:
#| eval: false

using MATLAB

mat"""
f = @() $A * $B;
timeit(f)
"""

LoadError: ArgumentError: Package MATLAB not found in current path.
- Run `import Pkg; Pkg.add("MATLAB")` to install the MATLAB package.

## Avoid memory allocation: some examples

1. Transposing matrix is an expensive memory operation.  
    - In R, the command 
        ```R
        t(A) %*% x
        ```
    will first transpose `A` then perform matrix multiplication, causing unnecessary memory allocation
    - Julia is smart to avoid transposing matrix if possible.

In [29]:
using Random, LinearAlgebra, BenchmarkTools
Random.seed!(123)

n = 1000
A = rand(n, n)
x = rand(n);

In [30]:
typeof(transpose(A))

Transpose{Float64, Matrix{Float64}}

In [31]:
fieldnames(typeof(transpose(A)))

(:parent,)

In [32]:
# same data in tranpose(A) and original matrix A
pointer(transpose(A).parent), pointer(A)

(Ptr{Float64} @0x0000000171408000, Ptr{Float64} @0x0000000171408000)

In [33]:
# dispatch to BLAS
# does *not* actually transpose the matrix
@benchmark transpose($A) * $x

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  22.708 μs … 93.083 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     23.708 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   23.940 μs ±  1.870 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄██▄▃ ▃▆▇▇▄▄▁             ▁▁▂▁▁▁▁▁ ▁                        ▂
  █████▇████████▇▄▆▅▅▅▄▆▆█▇███████████▇█▇▇█▇▆▄▆▄▄▅▄▅▄▅▃▅▅▆▄▄▅ █
  22.7 μs      Histogram: log(frequency) by time      31.1 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

In [34]:
# pre-allocate result
out = zeros(size(A, 2))
@benchmark mul!($out, transpose($A), $x)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  22.792 μs … 92.458 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     23.042 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   23.280 μs ±  1.450 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂██▆                                                        ▁
  █████▆▆▂▂▃▃▄▃▂▃▄▂▃▂▄▄▅▄▅▄▅▅▆▆▆▇▇▆█▇▇▆▅▆▆▆▄▅▅▄▄▄▃▃▄▄▄▅▄▄▄▂▄▅ █
  22.8 μs      Histogram: log(frequency) by time      29.2 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [35]:
# or call BLAS wrapper directly
@benchmark LinearAlgebra.BLAS.gemv!('T', 1.0, $A, $x, 0.0, $out)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  22.666 μs … 95.625 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     22.875 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   23.096 μs ±  1.433 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃█▆▂                                                        ▁
  ████▇▆▅▃▄▁▅▃▃▄▄▄▁▄▄▁▃▃▁▃▄▅▆▇▆▄▇▇▇▇▇▇▆▇▇▆▇▅▅▅▄▅▁▅▅▅▅▄▅▅▄▃▄▅▅ █
  22.7 μs      Histogram: log(frequency) by time        29 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [36]:
R"""
A <- $A
x <- $x
bench::mark(t(A) %*% x) %>%
  print(width = Inf)
""";

# A tibble: 1 × 13
  expression      min   median `itr/sec` mem_alloc `gc/sec` n_itr  n_gc
  <bch:expr> <bch:tm> <bch:tm>     <dbl> <bch:byt>    <dbl> <int> <dbl>
1 t(A) %*% x   2.02ms   2.13ms      458.    7.64MB     93.8   171    35
  total_time result            memory             time            
    <bch:tm> <list>            <list>             <list>          
1      373ms <dbl [1,000 × 1]> <Rprofmem [3 × 3]> <bench_tm [206]>
  gc                
  <list>            
1 <tibble [206 × 3]>


2. [Broadcasting](https://docs.julialang.org/en/v1/base/arrays/#Broadcast-and-vectorization-1) in Julia achieves vectorized code without creating intermediate arrays.

    Suppose we want to calculate elementsize maximum of absolute values of two large arrays. In R or Matlab, the command
```r
max(abs(X), abs(Y))
```
will create two intermediate arrays and then one result array.

In [37]:
using RCall

Random.seed!(123)
X, Y = rand(1000, 1000), rand(1000, 1000)

R"""
library(dplyr)
library(bench)
bench::mark(max(abs($X), abs($Y))) %>%
  print(width = Inf)
""";

# A tibble: 1 × 13
  expression                           min   median `itr/sec` mem_alloc `gc/sec`
  <bch:expr>                      <bch:tm> <bch:tm>     <dbl> <bch:byt>    <dbl>
1 max(abs(`#JL`$X), abs(`#JL`$Y))   8.08ms    8.2ms      121.    15.3MB     130.
  n_itr  n_gc total_time result    memory             time           
  <int> <dbl>   <bch:tm> <list>    <list>             <list>         
1    28    30      231ms <dbl [1]> <Rprofmem [3 × 3]> <bench_tm [58]>
  gc               
  <list>           
1 <tibble [58 × 3]>


In Julia, dot operations are fused so no intermediate arrays are created.

In [38]:
# no intermediate arrays created, only result array created
@benchmark max.(abs.($X), abs.($Y))

BenchmarkTools.Trial: 4598 samples with 1 evaluation.
 Range (min … max):  599.625 μs …   9.828 ms  ┊ GC (min … max):  0.00% … 88.07%
 Time  (median):     969.292 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.086 ms ± 640.504 μs  ┊ GC (mean ± σ):  12.57% ± 16.00%

  ▆   ▄█▇▇▅▂                                                    ▁
  █▆▁▁███████▄▄▁▁▁▁▁▁▅▆▄▁▄▇▇▇▇▇▆▇▅▅▆▅▃▃▃▃▄▅▆▇▆▄▁▇▅▅▅▅▅▃▆▁▃▆▅▅▆▆ █
  600 μs        Histogram: log(frequency) by time       4.33 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

Pre-allocating result array gets rid of memory allocation at all.

In [39]:
# no memory allocation at all!
Z = zeros(size(X)) # zero matrix of same size as X
@benchmark $Z .= max.(abs.($X), abs.($Y)) # .= (vs =) is important!

BenchmarkTools.Trial: 8109 samples with 1 evaluation.
 Range (min … max):  574.625 μs … 769.167 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     611.416 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   615.543 μs ±  11.792 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                        ▅█▇▆▆▅▄▄▄▄▃▃▃▃▃▂▂▂▂▁▂▁▁ ▁               ▂
  ▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▆▇▇██████████████████████████████▇▇█▇▆▆▆▆▆ █
  575 μs        Histogram: log(frequency) by time        662 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

**Exercise:** Annotate the broadcasting by `@turbo` and `@tturbo` (multi-threading) and benchmark again.

3. [View](https://docs.julialang.org/en/v1/base/arrays/#Views-(SubArrays-and-other-view-types)-1) avoids creating extra copy of matrix data.

In [40]:
Random.seed!(123)
A = randn(1000, 1000)

# sum entries in a sub-matrix
@benchmark sum($A[1:2:500, 1:2:500])

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  42.709 μs …   4.923 ms  ┊ GC (min … max):  0.00% … 97.99%
 Time  (median):     70.458 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   84.437 μs ± 232.927 μs  ┊ GC (mean ± σ):  16.38% ±  5.99%

                              ▅█    ▃                           
  ▂▁▁▄▄▂▂▂▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▄██▆▄▆███▆▇▇█▇▆▆▅▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂ ▃
  42.7 μs         Histogram: frequency by time           90 μs <

 Memory estimate: 488.36 KiB, allocs estimate: 2.

In [41]:
# view avoids creating a separate sub-matrix
# unfortunately it's much slower possibly because of cache misses
@benchmark sum(@view $A[1:2:500, 1:2:500])

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  239.875 μs … 304.083 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     240.084 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   243.280 μs ±   6.109 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █      ▅▃▂▂▁    ▁▂▁▁    ▁▁                                    ▁
  ██▆▇▇▄▂██████▅▅███████▇██████▇▇▆▆▇▇█▇▇▆▆▆▆▆▆▇▇▆▆▅▆▆▆▅▅▅▄▄▄▅▅▅ █
  240 μs        Histogram: log(frequency) by time        267 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

The [`@views`](https://docs.julialang.org/en/v1/base/arrays/#Base.@views) macro, which can be useful in [some operations](https://discourse.julialang.org/t/why-is-a-manual-in-place-addition-so-much-faster-than-and-on-range-indexed-arrays/3302).

In [42]:
@benchmark @views sum($A[1:2:500, 1:2:500])

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  239.875 μs … 302.416 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     240.042 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   241.736 μs ±   4.446 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █       ▂▄▁▁  ▁                                               ▁
  ██▅▄▅▆▄▂████▇▇█▆▄▂▆▇██▇▇▇▆▆▄▆▆▇▆▆▇▆▅▄▄▆▆▆▆▅▅▄▅▄▄▄▅▅▄▃▄▃▄▄▄▃▃▄ █
  240 μs        Histogram: log(frequency) by time        263 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.